학습방법 : Efficientnet-b7 모델을 사용하고 여러가지 Augmentation을 테스트 하였으며,  Kfold : 3로 나누어 진행하였고 epoch마다 모델을 저장하였습니다. accuracy와 loss를 고려하여 여러가지 모델을 앙상블 진행해보는 것이 목적이였으나, 런타임 중지로 인해 가장 높은 3가지 모델만 추가하여 앙상블을 진행하였습니다. 

Google Colab에서 진행.

In [ ]:
# Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

코랩 사용 환경 설정

In [ ]:
# 데이터 unzip, colab 사용 환경 설정
from google.colab import output
!cp "/content/drive/MyDrive/MNIST/235697_제 2회 컴퓨터 비전 학습 경진대회_data.zip" "data_2.zip"
!unzip "data_2.zip"
!mkdir "./dirty_mnist"
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
!mkdir "./test_dirty_mnist"
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
output.clear()

In [ ]:
# 필요한 모듈 install
!pip install pretrainedmodels
!pip install albumentations
!pip install --upgrade efficientnet-pytorch

In [ ]:
# 필요한 모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import imutils
import zipfile
import imgaug
import albumentations as A
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet50
from efficientnet_pytorch import EfficientNet
from albumentations.core.transforms_interface import ImageOnlyTransform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

seed, epoch, learning_rate 등 전역 변수 정의 

In [ ]:
# 전역 변수 정의
class config:
  seed = 2022
  device = "cuda"    
  num_classes = 26
  lr = 0.001
  epochs = 20
  train_batch_size = 16
  valid_batch_size = 8
  num_workers = 3
  folds = 3    
  IMG_SIZE = 256

Data 불러오기. 베이스라인 코드를 바탕으로 진행. 
Augmantation의 경우 Resize, RandomRotate90, RandomContrast 진행. 그밖에  다른 augmantation 다른 세션에서 진행하여 모델 저장.

In [ ]:
# Data 불러오기
dirty_mnist_answer = pd.read_csv("dirty_mnist_2nd_answer.csv")
namelist = os.listdir('./dirty_mnist/')

# numpy형식 이미지 Tensor 변환
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image.transpose((2, 0, 1))
        return {'image': torch.FloatTensor(image),
                'label': torch.FloatTensor(label)}

to_tensor = T.Compose([ ToTensor() ])

# 이미지 Dataset 변환 클래스
class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms= to_tensor,
                 augmentations = None
                 ):
        self.dir_path = dir_path
        self.meta_df = meta_df    
        self.transforms = transforms
        self.augmentations = augmentations

    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        image = cv2.imread(self.dir_path +\
                           str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                           cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, dsize=(config.IMG_SIZE, config.IMG_SIZE), interpolation=cv2.INTER_LINEAR)
        if self.augmentations:
            augmentations = A.Compose([
                A.Resize((224,224)),
                A.RandomRotation90(p=1),
                A.RandomContrast(),
            ])
            image = augmentations(image=image)['image']
        image = (image/255).astype('float')[..., np.newaxis]
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        sample = {'image': image, 'label': label}
        if self.transforms:
          sample = self.transforms(sample)
        return sample

model은 pre-tained된 efficientnet-b7으로 진행. Kfold 3으로 진행하였으나, 런타임 정지로 인해 여러개의 세션들에서 진행된 모델들 앙상블 진행.

In [ ]:
# 모델 학습
kfold = KFold(n_splits=config.folds, shuffle=True, random_state=config.seed)
best_models = []

# kfold 적용, val_acc가 개선되는 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(dirty_mnist_answer),1):
    print(f'[fold: {fold_index}]')
    torch.cuda.empty_cache()
    train_answer = dirty_mnist_answer.iloc[trn_idx]
    test_answer  = dirty_mnist_answer.iloc[val_idx]
    train_dataset = DatasetMNIST("dirty_mnist/", train_answer)
    valid_dataset = DatasetMNIST("dirty_mnist/", test_answer)

    train_data_loader = DataLoader(
        train_dataset,
        batch_size = config.train_batch_size,
        shuffle = False,
        num_workers = config.num_workers
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = config.valid_batch_size,
        shuffle = False,
        num_workers = config.num_workers
    )

    model = EfficientNet.from_pretrained('efficientnet-b7',in_channels = 1, num_classes= config.num_classes, dropout_rate = 0.3)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr = config.lr)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)
    criterion = nn.CrossEntropyLoss()
    valid_acc_max = 0

    for epoch in range(config.epochs):
        train_acc_list = []
        with tqdm(train_data_loader,
                total=train_data_loader.__len__(),
                unit="batch") as train_bar:
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)
                model.train()
                with torch.set_grad_enabled(True):
                    probs  = model(images)
                    loss = criterion(probs, labels)
                    loss.backward()
                    optimizer.step()

                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)

                train_bar.set_postfix(train_loss= loss.item(), train_acc = train_acc)
                
        valid_acc_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)
                model.eval()
                with torch.no_grad():
                    probs  = model(images)
                    valid_loss = criterion(probs, labels)

                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    valid_acc_list.append(batch_acc)

                valid_acc = np.mean(valid_acc_list)
                valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                      valid_acc = valid_acc)
            
        lr_scheduler.step()

        if valid_acc_max < valid_acc:
            valid_acc_max = valid_acc
            best_model = model
            MODEL = "EfficientnetB7"
            path = "/content/drive/MyDrive/MNIST/models/"
            torch.save(best_model, f'{path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth')

    best_models.append(best_model)

In [ ]:
# 학습 결과 확인
sample_images = images.cpu().detach().numpy()
sample_prob = probs
sample_labels = labels

idx = 1
plt.imshow(sample_images[idx][0])
plt.title("sample input image")
plt.show()

print('예측값 : ',dirty_mnist_answer.columns[1:][sample_prob[idx] > 0.5])
print('정답값 : ', dirty_mnist_answer.columns[1:][sample_labels[idx] > 0.5])

In [ ]:
# test Dataset 정의
sample_submission = pd.read_csv("sample_submission.csv")
test_dataset = DatasetMNIST("test_dirty_mnist/", sample_submission)
batch_size = 16
test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

여러개의 세션에서 진행한 모델중 valid-accuracy와 loss를 고려하여 best_models에 3개의 모델 추가

In [ ]:
# 다른 세션에서 파라미터 튜닝을 통해 저장한 model best_models에 불러옴
model1 = torch.load('/content/drive/MyDrive/MNIST/models/1_EfficientnetB7_0.5067_epoch_19.pth')
model2 = torch.load('/content/drive/MyDrive/MNIST/models/2_EfficientnetB7_0.5641_epoch_19.pth')
model3 = torch.load('/content/drive/MyDrive/MNIST/models/1_EfiicientnetB7_0.5919_epoch_12.pth')

best_models.append(model1)
best_models.append(model2)
best_models.append(model3)

best_models에 있는 model들 앙상블 진행.

In [ ]:
# 앙상블 적용
predictions_list = []
prediction_df = pd.read_csv("sample_submission.csv")

for model in best_models:
    prediction_array = np.zeros([prediction_df.shape[0],
                                 prediction_df.shape[1] -1])
    for idx, sample in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs  = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0],:]\
                         = preds.astype(int)
                         
    predictions_list.append(prediction_array[...,np.newaxis])

predictions_array = np.concatenate(predictions_list, axis = 2)
predictions_mean = predictions_array.mean(axis = 2)
predictions_mean = (predictions_mean > 0.5) * 1
predictions_mean[1,:]

In [ ]:
# 제출 파일 생성
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:,1:] = predictions_mean
sample_submission.to_csv("MNIST_prob.csv", index = False)
sample_submission